# Gypsum-DLでSBVSの前処理をおこなう

## Gypsum-DLとは
gypsum-dl はDurrant研が開発し報告した分子の前処理用のパッケージです。類似の前処理用のソフトウェアとしてはOpenEye’s OMEGA/QUACPAC や Schrödingerの LigPrep等が有名ですが、OSSでProtonationを考慮した前処理ができるツールはあまりありません(Openbabelくらい)。Gypsum-DLはRDKitをベースにしたツールであり環構造の多様性も加味した配座発生、脱塩なども可能です

注) 企業の研究環境では、OpenEyeやSchrödingerの製品を利用することが多いと思いますが、今回のセッションではgypsum-dlを利用します。 

### 参考情報
- [原著論文](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-019-0358-3)
- [GitHub repo](https://github.com/durrantlab/gypsum_dl)

## 動作確認を行う
後ほどMaizeで利用しますが、その前に直接Gypsumを動かしてみましょう。本ディレクトリにかんたんなスクリプトを置きました。

In [ ]:
!cat run_gypsum_example.sh

### コマンドラインから実行する例

```bash
$ conda activate gypsum
$ cd [gypsum repo dir]
$ python run_gypsum_dl.py -s ../cheminfo_tutorial_20241028/data/sample_molecules.smi -o ./gypsum_out --add_html_output
$ open gypsum_out/gypsum_dl_success.html #webブラウザで結果を確認
```

## maizeの設定ファイルにGypsumとAutoDoc Vinaを追加する

maizeはmaize.tomlというファイルに各種の設定を追記することで外部プログラムを実行することができます。
- デフォルトではXDG_CONFIG_HOME以下にあるmaize.tomlが参照されますが、もしXDG_CONFIG_HOMEが環境変数として設定されていない場合[警告が出ます](https://stackoverflow.com/questions/3373948/equivalents-of-xdg-config-home-and-xdg-data-home-on-mac-os-x)。

### ハンズオン用に設定ファイルを作成する

gypsumのpathは上記で動作確認を行ったものと同じです。
AutoDock Vinaのpathは01で動作確認したものですが、vinaのpythonのpathはmaize仮想環境のものを設定してください。

```bash
#just example
!cat../data/maize.toml
[vina]
python = "/home/iwatobipen/miniforge3/envs/maize-dev/bin/python"
commands.vina = "/opt/vina/bin/vina"

[gypsum]
scripts.gypsum.interpreter = "/home/iwatobipen/miniforge3/envs/gypsum/bin/python"
scripts.gypsum.location = "/home/iwatobipen/dev/gypsum_dl/run_gypsum_dl.py"
```

## 設定ファイルを有効にしてjupyte notebookを再起動する

上記のmaize.tomlを有効にするために環境変数を設定してからjupyter notebookを再起動します。

```sh
$ export XDG_CONFIG_HOME="[各自のpath]/cheminfo_tutorial_20241028/data"
$ jupyter notebook
```

#### 補足情報

もしデフォルトのXDG_CONFIG_HOME/maize.toml以外のファイルを設定ファイルとして利用する場合は以下のようにプログラム内で
読み込みます。

```python
from maize.utilities.io import Config, NodeConfig
flow.config = Config()
flow.config.update(Path("path/to/config.toml"))
```

以下のコードは設定ファイルの上書きで設定が更新されていることを確かめています。
- [参考](https://molecularai.github.io/maize/docs/userguide.html)

In [ ]:
# 設定前後のConfigのチェック
from pathlib import Path
from maize.utilities.io import Config, NodeConfig
from maize.core.workflow import Workflow
from pprint import pprint

flow = Workflow()
flow.config = Config()
pprint(flow.config)

In [ ]:
flow.config.update(Path("../data/maize.toml"))
pprint(flow.config)